<a href="https://colab.research.google.com/github/h4nkyn/COVID-19-Simulator/blob/main/Kopie_von_Kopie_von_Interacting_with_Jukebox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTANT NOTE ON SYSTEM REQUIREMENTS:

Old text: [If you are connecting to a hosted runtime, make sure it has a P100 GPU (optionally run !nvidia-smi to confirm). Go to Edit>Notebook Settings to set this.

CoLab may first assign you a lower memory machine if you are using a hosted runtime.  If so, the first time you try to load the 5B model, it will run out of memory, and then you'll be prompted to restart with more memory (then return to the top of this CoLab).  If you continue to have memory issues after this (or run into issues on your own home setup), switch to the 1B model.

If you are using a local GPU, we recommend V100 or P100 with 16GB GPU memory for best performance. For GPU’s with less memory, we recommend using the 1B model and a smaller batch size throughout.]  

Edit: If you're a free Colab user, check which GPU you were assigned in the cell below. If you get a T4, you can run the 5b_lyrics model. The K80 can't handle it unfortunately, so you might have to switch to the weaker 1b_lyrics model.



In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-1732cdf2-36bb-fb07-a75b-6ba7a4fc862f)


Mount Google Drive to save sample levels as they are generated.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Prepare the environment.

In [ ]:
!pip install torch==1.9.1+cu111 torchvision==0.10.1+cu111 torchaudio==0.9.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install git+https://github.com/craftmine1000/jukebox-opt.git

  Cloning https://github.com/craftmine1000/jukebox-opt.git to /tmp/pip-req-build-d5tkvf29
  Running command git clone -q https://github.com/craftmine1000/jukebox-opt.git /tmp/pip-req-build-d5tkvf29
     |████████████████████████████████| 60 kB 6.4 MB/s 
     |████████████████████████████████| 238 kB 17.1 MB/s 
     |████████████████████████████████| 3.5 MB 51.0 MB/s 
     |████████████████████████████████| 1.6 MB 48.1 MB/s 
     |████████████████████████████████| 2.4 MB 52.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 20.2 MB 1.2 MB/s 
  Created wheel for jukebox: filename=jukebox-1.0-py3-none-any.whl size=204812 sha256=1113bc8ea6dfc839583fc20307d1b176d46c19e9c37bd4db1d14e7e3fe9063da
  Stored in directory: /tmp/pip-ephem-wheel-cache-h3diy0a8/wheels/51/84/29/89b70e25b158ee036f2f49f3302ba9b9f9216bbd1be01cd8d7
  Created wheel for fire: filename=fire-0.1.3-py2.py3

In [ ]:
import jukebox
import torch as t
import librosa
import os
from IPython.display import Audio
from jukebox.make_models import make_vqvae, make_prior, MODELS, make_model
from jukebox.hparams import Hyperparams, setup_hparams
from jukebox.sample import sample_single_window, _sample, \
                           sample_partial_window, upsample, \
                           load_prompts
from jukebox.utils.dist_utils import setup_dist_from_mpi
from jukebox.utils.torch_utils import empty_cache
rank, local_rank, device = setup_dist_from_mpi()

Welcome to jukebox-opt, a system ram optimized version of jukebox.
For most notebooks/enviroments this acts as a mostly drop in replacement.

I say mostly because most notebooks/enviroments tend to load the tokens (zs)
onto the gpu when i want them on the cpu to allow for longer songs, my code
will put it back once loaded from a checkpoint, if the song is to longer than normal
when loading it will use too much gpu memory and probably error out later in
generation. To fix this, go through your notebook and change all 'cuda' to 'cpu'
and all .cuda() to .cpu() for the tokens (zs).

Example; change "zs = t.load(blablabla, location='cuda')" to "zs = t.load(blablabla, location='cpu')"
Example; change "zs = [ z.cuda() for z in zs ]" to "zs = [ z.cpu() for z in zs ]"
Example; change "zs[blabla].cuda()" to "zs[blabla].cpu()"


Using cuda True


# Sample from the 5B or 1B Lyrics Model



In [ ]:
model = '5b_lyrics' # or '5b' or '1b_lyrics'
hps = Hyperparams()
hps.sr = 44100
hps.n_samples = 2
# Specifies the directory to save the sample in.
# We set this to the Google Drive mount point.
hps.name = '/content/gdrive/My Drive/sampleselvisbottomjeans'
chunk_size = 16 if model in ('5b', '5b_lyrics') else 32
max_batch_size = 2 if model in ('5b', '5b_lyrics') else 16
hps.levels = 3
hps.hop_fraction = [1,1,.125]

vqvae, *priors = MODELS[model]
vqvae = make_vqvae(setup_hparams(vqvae, dict(sample_length = 1048576)), device)
top_prior = make_prior(setup_hparams(priors[-1], dict()), vqvae, device)

Running  wget -O /root/.cache/jukebox/models/5b/vqvae.pth.tar https://openaipublic.azureedge.net/jukebox/models/5b/vqvae.pth.tar


100%|██████████| 669/669 [00:00<00:00, 63369.23it/s]


Restored from /root/.cache/jukebox/models/5b/vqvae.pth.tar
0: Loading vqvae in eval mode


100%|██████████| 79/79 [00:40<00:00,  1.97it/s]


Loading artist IDs from /usr/local/lib/python3.7/dist-packages/jukebox/data/ids/v2_artist_ids.txt
Loading artist IDs from /usr/local/lib/python3.7/dist-packages/jukebox/data/ids/v2_genre_ids.txt
Level:2, Cond downsample:None, Raw to tokens:128, Sample length:1048576
Running  wget -O /root/.cache/jukebox/models/5b_lyrics/prior_level_2.pth.tar https://openaipublic.azureedge.net/jukebox/models/5b_lyrics/prior_level_2.pth.tar


100%|██████████| 1194/1194 [10:20<00:00,  1.92it/s]


Restored from /root/.cache/jukebox/models/5b_lyrics/prior_level_2.pth.tar
0: Loading prior in eval mode


# Select mode
Run one of these cells to select the desired mode.

In [ ]:
# The default mode of operation.
# Creates songs based on artist and genre conditioning.
mode = 'ancestral'
codes_file=None
audio_file=None
prompt_length_in_seconds=None

In [ ]:
# Prime song creation using an arbitrary audio sample.
mode = 'primed'
codes_file=None
# Specify an audio file here.
audio_file = '/content/gdrive/My Drive/primer.wav'
# Specify how many seconds of audio to prime on.
prompt_length_in_seconds=6

Run this cell to automatically resume from the latest checkpoint file, but only if the checkpoint file exists.
This will override the selected mode.
We will assume the existance of a checkpoint means generation is complete and it's time for upsamping to occur.

In [ ]:
if os.path.exists(hps.name):
  # Identify the lowest level generated and continue from there.
  for level in [0, 1, 2]:
    data = f"{hps.name}/level_{level}/data.pth.tar"
    if os.path.isfile(data):
      mode = 'upsample'
      codes_file = data
      print('Upsampling from level '+str(level))
      break
print('mode is now '+mode)

Run the cell below regardless of which mode you chose.

In [ ]:
sample_hps = Hyperparams(dict(mode=mode, codes_file=codes_file, audio_file=audio_file, prompt_length_in_seconds=prompt_length_in_seconds))

Specify your choice of artist, genre, lyrics, and length of musical sample. 

In [ ]:
sample_length_in_seconds = 90          # Full length of musical sample to generate - we find songs in the 1 to 4 minute
                                       # range work well, with generation time proportional to sample length.  
                                       # This total length affects how quickly the model 
                                       # progresses through lyrics (model also generates differently
                                       # depending on if it thinks it's in the beginning, middle, or end of sample)
hps.sample_length = (int(sample_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
assert hps.sample_length >= top_prior.n_ctx*top_prior.raw_to_tokens, f'Please choose a larger sampling rate'

In [ ]:
# Note: Metas can contain different prompts per sample.
# By default, all samples use the same prompt.
metas = [dict(artist = "elvis presley",
            genre = "rock n roll",
            total_length = hps.sample_length,
            offset = 0,
            lyrics = """Shawty had them Apple Bottom jeans (Jeans), boots with the fur (With the fur)
The whole club was lookin' at her
She hit the flo' (She hit the flo'), next thing you know
Shawty got low-low-low-low-low-low-low-low
Them baggy sweat pants and the Reeboks with the straps (With the straps)
She turned around and gave that big booty a smack (Ayy!)
She hit the flo' (She hit the flo'), next thing you know
Shawty got low-low-low-low-low-low-low-low

Hey, I ain't never seen nothin' that'll make me go
This crazy all night spendin' my dough
Had a million dollar vibe and a bottle to go
Them birthday cakes they stole the show
So sexual, she was flexible
Professional, drinkin' X and O
Hold up, wait a minute, do I see what I think I, whoa
Did I think I see shawty get low?
""",
            ),
          ] * hps.n_samples
labels = [None, None, top_prior.labeller.get_batch_labels(metas, 'cuda')]

Optionally adjust the sampling temperature (we've found .98 or .99 to be our favorite).  


In [ ]:
sampling_temperature = .98

lower_batch_size = 16
max_batch_size = 3 if model in ('5b', '5b_lyrics') else 16
lower_level_chunk_size = 32
chunk_size = 16 if model in ('5b', '5b_lyrics') else 32
sampling_kwargs = [dict(temp=.99, fp16=True, max_batch_size=lower_batch_size,
                        chunk_size=lower_level_chunk_size),
                    dict(temp=0.99, fp16=True, max_batch_size=lower_batch_size,
                         chunk_size=lower_level_chunk_size),
                    dict(temp=sampling_temperature, fp16=True, 
                         max_batch_size=max_batch_size, chunk_size=chunk_size)]

Now we're ready to sample from the model. We'll generate the top level (2) first, followed by the first upsampling (level 1), and the second upsampling (0).  In this CoLab we load the top prior separately from the upsamplers, because of memory concerns on the hosted runtimes. If you are using a local machine, you can also load all models directly with make_models, and then use sample.py's ancestral_sampling to put this all in one step.

After each level, we decode to raw audio and save the audio files.   

This next cell will take a while (approximately 10 minutes per 20 seconds of music sample)

In [ ]:
if sample_hps.mode == 'ancestral':
  zs = [t.zeros(hps.n_samples,0,dtype=t.long, device='cpu') for _ in range(len(priors))]
  zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
elif sample_hps.mode == 'upsample':
  assert sample_hps.codes_file is not None
  # Load codes.
  data = t.load(sample_hps.codes_file, map_location='cpu')
  zs = [z.cpu() for z in data['zs']]
  assert zs[-1].shape[0] == hps.n_samples, f"Expected bs = {hps.n_samples}, got {zs[-1].shape[0]}"
  del data
  print('Falling through to the upsample step later in the notebook.')
elif sample_hps.mode == 'primed':
  assert sample_hps.audio_file is not None
  audio_files = sample_hps.audio_file.split(',')
  duration = (int(sample_hps.prompt_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
  x = load_prompts(audio_files, duration, hps)
  zs = top_prior.encode(x, start_level=0, end_level=len(priors), bs_chunks=x.shape[0])
  zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
else:
  raise ValueError(f'Unknown sample mode {sample_hps.mode}.')

Sampling level 2
Sampling 8192 tokens for [0,8192]. Conditioning on 0 tokens
Ancestral sampling 2 samples with temp=0.98, top_k=0, top_p=0.0
8192/8192 [10:39<00:00, 12.80it/s]
Sampling 8192 tokens for [1024,9216]. Conditioning on 7168 tokens
Primed sampling 2 samples with temp=0.98, top_k=0, top_p=0.0
448/448 [00:56<00:00,  7.97it/s]
1024/1024 [01:35<00:00, 10.77it/s]
Sampling 8192 tokens for [2048,10240]. Conditioning on 7168 tokens
Primed sampling 2 samples with temp=0.98, top_k=0, top_p=0.0
448/448 [00:56<00:00,  7.99it/s]
1024/1024 [01:34<00:00, 10.79it/s]
Sampling 8192 tokens for [3072,11264]. Conditioning on 7168 tokens
Primed sampling 2 samples with temp=0.98, top_k=0, top_p=0.0
448/448 [00:55<00:00,  8.00it/s]
1024/1024 [01:34<00:00, 10.78it/s]
Sampling 8192 tokens for [4096,12288]. Conditioning on 7168 tokens
Primed sampling 2 samples with temp=0.98, top_k=0, top_p=0.0
448/448 [00:56<00:00,  7.99it/s]
1024/1024 [01:34<00:00, 10.79it/s]
Sampling 8192 tokens for [5120,13312]. Co

Listen to the results from the top level (note this will sound very noisy until we do the upsampling stage).  You may have more generated samples, depending on the batch size you requested.

In [ ]:
Audio(f'{hps.name}/level_2/item_0.wav')

We are now done with the large top_prior model, and instead load the upsamplers.

In [ ]:
# Set this False if you are on a local machine that has enough memory (this allows you to do the
# lyrics alignment visualization during the upsampling stage). For a hosted runtime, 
# we'll need to go ahead and delete the top_prior if you are using the 5b_lyrics model.
if True:
  del top_prior
  empty_cache()
  top_prior=None
upsamplers = [make_prior(setup_hparams(prior, dict()), vqvae, 'cpu') for prior in priors[:-1]]
labels[:2] = [prior.labeller.get_batch_labels(metas, 'cuda') for prior in upsamplers]

Conditioning on 1 above level(s)
Checkpointing convs
Checkpointing convs


100%|██████████| 72/72 [00:07<00:00,  9.93it/s]


Loading artist IDs from /usr/local/lib/python3.7/dist-packages/jukebox/data/ids/v2_artist_ids.txt
Loading artist IDs from /usr/local/lib/python3.7/dist-packages/jukebox/data/ids/v2_genre_ids.txt
Level:0, Cond downsample:4, Raw to tokens:8, Sample length:65536
Running  wget -O /root/.cache/jukebox/models/5b/prior_level_0.pth.tar https://openaipublic.azureedge.net/jukebox/models/5b/prior_level_0.pth.tar


100%|██████████| 1005/1005 [01:29<00:00, 11.27it/s]


Restored from /root/.cache/jukebox/models/5b/prior_level_0.pth.tar
0: Loading prior in eval mode
Conditioning on 1 above level(s)
Checkpointing convs
Checkpointing convs


100%|██████████| 72/72 [00:06<00:00, 10.66it/s]


Loading artist IDs from /usr/local/lib/python3.7/dist-packages/jukebox/data/ids/v2_artist_ids.txt
Loading artist IDs from /usr/local/lib/python3.7/dist-packages/jukebox/data/ids/v2_genre_ids.txt
Level:1, Cond downsample:4, Raw to tokens:32, Sample length:262144
Running  wget -O /root/.cache/jukebox/models/5b/prior_level_1.pth.tar https://openaipublic.azureedge.net/jukebox/models/5b/prior_level_1.pth.tar


100%|██████████| 1005/1005 [01:45<00:00,  9.56it/s]


Restored from /root/.cache/jukebox/models/5b/prior_level_1.pth.tar
0: Loading prior in eval mode


Please note: this next upsampling step will take several hours.  At the free tier, Google CoLab lets you run for 12 hours.  As the upsampling is completed, samples will appear in the Files tab (you can access this at the left of the CoLab), under "samples" (or whatever hps.name is currently).  Level 1 is the partially upsampled version, and then Level 0 is fully completed.

In [ ]:
zs = upsample(zs, labels, sampling_kwargs, [*upsamplers, top_prior], hps)

Listen to your final sample!

In [ ]:
Audio(f'{hps.name}/level_0/item_0.wav')

In [ ]:
del upsamplers
empty_cache()

# Co-Composing with the 5B or 1B Lyrics Model

For more control over the generations, try co-composing with either the 5B or 1B Lyrics Models.  Again, specify your artist, genre, and lyrics. However, now instead of generating the entire sample, the model will return 3 short options for the opening of the piece (or up to 16 options if you use the 1B model instead).  Choose your favorite, and then continue the loop, for as long as you like.  Throughout these steps, you'll be listening to the audio at the top prior level, which means it will sound quite noisy.  When you are satisfied with your co-creation, continue on through the upsampling section. This will render the piece in higher audio quality.

NOTE: CoLab will first assign you a lower memory machine if you are using a hosted runtime.  The next cell will run out of memory, and then you'll be prompted to restart with more memory (then return to the top of this CoLab).  If you continue to have memory issues after this (or run into issues on your own home setup), switch to the 1B model. 

In [ ]:
model = "5b_lyrics" # or "1b_lyrics"
hps = Hyperparams()
hps.sr = 44100
hps.n_samples = 3 if model in ('5b', '5b_lyrics') else 16
# Specifies the directory to save the sample in.
# We set this to the Google Drive mount point.
hps.name = '/content/gdrive/My Drive/co_composer'
hps.sample_length = 1048576 if model in ('5b', '5b_lyrics') else 786432 
chunk_size = 16 if model in ('5b', '5b_lyrics') else 32
max_batch_size = 3 if model in ('5b', '5b_lyrics') else 16
hps.hop_fraction = [.5, .5, .125] 
hps.levels = 3

vqvae, *priors = MODELS[model]
vqvae = make_vqvae(setup_hparams(vqvae, dict(sample_length = hps.sample_length)), device)
top_prior = make_prior(setup_hparams(priors[-1], dict()), vqvae, device)

# Select mode
Run one of these cells to select the desired mode.

In [ ]:
# The default mode of operation.
# Creates songs based on artist and genre conditioning.
mode = 'ancestral'
codes_file=None
audio_file=None
prompt_length_in_seconds=None

In [ ]:
# Prime song creation using an arbitrary audio sample.
mode = 'primed'
codes_file=None
# Specify an audio file here.
audio_file = '/content/gdrive/My Drive/primer.wav'
# Specify how many seconds of audio to prime on.
prompt_length_in_seconds=12

Run the cell below regardless of which mode you chose.

In [ ]:
sample_hps = Hyperparams(dict(mode=mode, codes_file=codes_file, audio_file=audio_file, prompt_length_in_seconds=prompt_length_in_seconds))

Specify your choice of artist, genre, lyrics, and length of musical sample. 

In [ ]:
sample_length_in_seconds = 71          # Full length of musical sample to generate - we find songs in the 1 to 4 minute
                                       # range work well, with generation time proportional to sample length.  
                                       # This total length affects how quickly the model 
                                       # progresses through lyrics (model also generates differently
                                       # depending on if it thinks it's in the beginning, middle, or end of sample)
hps.sample_length = (int(sample_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
assert hps.sample_length >= top_prior.n_ctx*top_prior.raw_to_tokens, f'Please choose a larger sampling rate'

In [ ]:
metas = [dict(artist = "Zac Brown Band",
            genre = "Country",
            total_length = hps.sample_length,
            offset = 0,
            lyrics = """I met a traveller from an antique land,
            Who said—“Two vast and trunkless legs of stone
            Stand in the desert. . . . Near them, on the sand,
            Half sunk a shattered visage lies, whose frown,
            And wrinkled lip, and sneer of cold command,
            Tell that its sculptor well those passions read
            Which yet survive, stamped on these lifeless things,
            The hand that mocked them, and the heart that fed;
            And on the pedestal, these words appear:
            My name is Ozymandias, King of Kings;
            Look on my Works, ye Mighty, and despair!
            Nothing beside remains. Round the decay
            Of that colossal Wreck, boundless and bare
            The lone and level sands stretch far away
            """,
            ),
          ] * hps.n_samples
labels = top_prior.labeller.get_batch_labels(metas, 'cuda')

## Generate 3 options for the start of the song

Initial generation is set to be 4 seconds long, but feel free to change this

In [ ]:
def seconds_to_tokens(sec, sr, prior, chunk_size):
  tokens = sec * hps.sr // prior.raw_to_tokens
  tokens = ((tokens // chunk_size) + 1) * chunk_size
  assert tokens <= prior.n_ctx, 'Choose a shorter generation length to stay within the top prior context'
  return tokens

In [ ]:
initial_generation_in_seconds = 4
tokens_to_sample = seconds_to_tokens(initial_generation_in_seconds, hps.sr, top_prior, chunk_size)

Change the sampling temperature if you like (higher is more random).  Our favorite is in the range .98 to .995

In [ ]:
sampling_temperature = .98

lower_batch_size = 16
max_batch_size = 3 if model in ('5b', '5b_lyrics') else 16
lower_level_chunk_size = 32
chunk_size = 16 if model in ('5b', '5b_lyrics') else 32
sampling_kwargs = dict(temp=sampling_temperature, fp16=True, max_batch_size=lower_batch_size,
                        chunk_size=lower_level_chunk_size)

In [ ]:
if sample_hps.mode == 'ancestral':
  zs=[t.zeros(hps.n_samples,0,dtype=t.long, device='cuda') for _ in range(3)]
  zs=sample_partial_window(zs, labels, sampling_kwargs, 2, top_prior, tokens_to_sample, hps)
elif sample_hps.mode == 'primed':
  assert sample_hps.audio_file is not None
  audio_files = sample_hps.audio_file.split(',')
  duration = (int(sample_hps.prompt_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
  x = load_prompts(audio_files, duration, hps)
  zs = top_prior.encode(x, start_level=0, end_level=len(priors), bs_chunks=x.shape[0])
  zs = sample_partial_window(zs, labels, sampling_kwargs, 2, top_prior, tokens_to_sample, hps)
x = vqvae.decode(zs[2:], start_level=2).cpu().numpy()

Listen to your generated samples, and then pick a favorite. If you don't like any, go back and rerun the cell above. 

** NOTE this is at the noisy top level, upsample fully (in the next section) to hear the final audio version

In [ ]:
for i in range(hps.n_samples):
  librosa.output.write_wav(f'noisy_top_level_generation_{i}.wav', x[i], sr=44100)

In [ ]:
Audio('noisy_top_level_generation_0.wav')

In [ ]:
Audio('noisy_top_level_generation_1.wav')

In [ ]:
Audio('noisy_top_level_generation_2.wav')

If you don't like any of the options, return a few cells back to "Sample a few options..." and rerun from there.

## Choose your favorite sample and request longer generation

---

(Repeat from here)


In [ ]:
my_choice=0

In [ ]:
zs[2]=zs[2][my_choice].repeat(hps.n_samples,1)
t.save(zs, 'zs-checkpoint2.t')

In [ ]:
# Set to True to load the previous checkpoint:
if False:
  zs=t.load('zs-checkpoint2.t') 

Choose the length of the continuation.  The 1B model can generate up to 17 second samples and the 5B up to 23 seconds, but you'll want to pick a shorter continuation length so that it will be able to look back at what you've generated already.  Here we've chosen 4 seconds.

In [ ]:
continue_generation_in_seconds=4
tokens_to_sample = seconds_to_tokens(continue_generation_in_seconds, hps.sr, top_prior, chunk_size)

The next step asks the top prior to generate more of the sample. It'll take up to a few minutes, depending on the sample length you request.

In [ ]:
zs = sample_partial_window(zs, labels, sampling_kwargs, 2, top_prior, tokens_to_sample, hps)
x = vqvae.decode(zs[2:], start_level=2).cpu().numpy()

Now listen to the longer versions of the sample you selected, and again choose a favorite sample.  If you don't like any, return back to the cell where you can load the checkpoint, and continue again from there.

When the samples start getting long, you might not always want to listen from the start, so change the playback start time later on if you like.

In [ ]:
playback_start_time_in_seconds = 0 

In [ ]:
for i in range(hps.n_samples):
  librosa.output.write_wav(f'top_level_continuation_{i}.wav', x[i][playback_start_time_in_seconds*44100:], sr=44100)

In [ ]:
Audio('top_level_continuation_0.wav')

In [ ]:
Audio('top_level_continuation_1.wav')

In [ ]:
Audio('top_level_continuation_2.wav')

To make a longer song, return back to "Choose your favorite sample" and loop through that again

# Upsample Co-Composition to Higher Audio Quality

Choose your favorite sample from your latest group of generations.  (If you haven't already gone through the Co-Composition block, make sure to do that first so you have a generation to upsample).

In [ ]:
choice = 0
select_best_sample = True  # Set false if you want to upsample all your samples 
                           # upsampling sometimes yields subtly different results on multiple runs,
                           # so this way you can choose your favorite upsampling

In [ ]:
if select_best_sample:
  zs[2]=zs[2][choice].repeat(zs[2].shape[0],1)

t.save(zs, 'zs-top-level-final.t')

Note: If you are using a CoLab hosted runtime on the free tier, you may want to download this zs-top-level-final.t file, and then restart an instance and load it in the next cell.  The free tier will last a maximum of 12 hours, and the upsampling stage can take many hours, depending on how long a sample you have generated.

In [ ]:
if False:
  zs = t.load('zs-top-level-final.t')

assert zs[2].shape[1]>=2048, f'Please first generate at least 2048 tokens at the top level, currently you have {zs[2].shape[1]}'
hps.sample_length = zs[2].shape[1]*top_prior.raw_to_tokens

In [ ]:
# Set this False if you are on a local machine that has enough memory (this allows you to do the
# lyrics alignment visualization). For a hosted runtime, we'll need to go ahead and delete the top_prior
# if you are using the 5b_lyrics model.
if True:
  del top_prior
  empty_cache()
  top_prior=None

upsamplers = [make_prior(setup_hparams(prior, dict()), vqvae, 'cpu') for prior in priors[:-1]]

In [ ]:
sampling_kwargs = [dict(temp=.99, fp16=True, max_batch_size=16, chunk_size=32),
                    dict(temp=0.99, fp16=True, max_batch_size=16, chunk_size=32),
                    None]

if type(labels)==dict:
  labels = [prior.labeller.get_batch_labels(metas, 'cuda') for prior in upsamplers] + [labels] 

This next step upsamples 2 levels.  The level_1 samples will be available after around one hour (depending on the length of your sample) and are saved under {hps.name}/level_0/item_0.wav, while the fully upsampled level_0 will likely take 4-12 hours. You can access the wav files down below, or using the "Files" panel at the left of this CoLab.

(Please note, if you are using this CoLab on Google's free tier, you may want to download intermediate steps as the connection will last for a maximum 12 hours.)

In [ ]:
zs = upsample(zs, labels, sampling_kwargs, [*upsamplers, top_prior], hps)

In [ ]:
Audio(f'{hps.name}/level_0/item_0.wav')